# Benchmark for d-wave

In [ ]:
from typing import Dict, Tuple, Set, List, Any

In [ ]:
import matplotlib.pylab as plt

In [ ]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [ ]:
base_params = {
    "answer_mode": "raw",
    "auto_scale": True,
    "num_reads": 1,  # raw will dump out all results
}
sampler = DWaveSampler(solver="DW_2000Q_5")
embedding = EmbeddingComposite(sampler)

In [ ]:
from qlp.mds import graph_tools as gt
from qlp.mds.qubo import get_mds_qubo
from qlp.mds.solver import classical_search, mds_schedule_submit

# Examples

## Most simple graph with unique sol

In [ ]:
graph = {(1, 0,), (2, 0)}

gt.get_plot(graph)
plt.show()

In [ ]:
qubo = get_mds_qubo(graph)
qubo.todense()

### Solve problem on D-Wave

In [ ]:
e, nodes = classical_search(qubo)
print("Brute force search classical minimal energy", e)
gt.get_plot(graph, nodes[0])
plt.show()

In [ ]:
help(mds_schedule_submit)

In [ ]:
p_schedules = [
    {"penalty": 1, "schedule": [(0, 0.0), (400, 1.0)]},
    {"penalty": 9, "schedule": [(0, 1.0), (200, 0.5), (400, 0.5), (600, 1.0)]},
]

In [ ]:
results = mds_schedule_submit(graph, p_schedules, embedding)

In [ ]:
results